<a href="https://colab.research.google.com/github/Manamstn/CAP5610-ML/blob/main/HW3_Titanic_DataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question-4: Build decision tree and random forest using Scikit Learn

In [ ]:
#MANA MOSTAANI
#Homework 3
#Please upload the titanic data set

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
#reading the data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

1 and 2. Data Preprocessing and Feature Selection ***(for train dataset)***

---



In [ ]:
#Data Cleaning
#Checking the missing values

train_data.isnull().sum()
train_data_cpy = train_data.copy()
#remove cabin column (Preprocessing stage)
train_data_cpy.drop('Cabin', axis=1,inplace=True)

#remove the rows that age and embarked have 0 values
train_data_cpy.dropna(subset=['Age','Embarked'], inplace=True)
#remove PassengerId and Ticket columns
train_data_cpy.drop('PassengerId', axis=1,inplace=True)
train_data_cpy.drop('Ticket', axis=1,inplace=True)

#remove Name, and Embarked
train_data_cpy.drop('Name', axis=1,inplace=True)
train_data_cpy.drop('Embarked', axis=1,inplace=True)
#Turning the Sex Type into 0 and 1 for Female and Male respectively
train_data_cpy["Sex"] = train_data_cpy["Sex"].map({"female":1, "male":0})
#dividing Age column by 10 to have Age groups
train_data_cpy["Age"] = pd.cut(train_data_cpy["Age"],bins=[0,10,20,30,40,50,60,70,80,90,100],labels=[0,1,2,3,4,5,6,7,8,9])
#we should remove the Fare that are more than 300
train_data_cpy = train_data_cpy[train_data_cpy['Fare'] <= 300]
train_data_cpy["Age"] = train_data_cpy["Age"].astype("int")

In [ ]:
#doing Data Correlation to select the relevant features in our dataset
train_data_cpy.corr()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
Survived,1.000000,-0.351579,0.539865,-0.078226,-0.012733,0.096227,0.272376
Pclass,-0.351579,1.000000,-0.152241,-0.353602,0.062048,0.023059,-0.616062
Sex,0.539865,-0.152241,1.000000,-0.101400,0.106433,0.251331,0.228917
Age,-0.078226,-0.353602,-0.101400,1.000000,-0.294708,-0.178799,0.095978
SibSp,-0.012733,0.062048,0.106433,-0.294708,1.000000,0.383571,0.199056
Parch,0.096227,0.023059,0.251331,-0.178799,0.383571,1.000000,0.260993
Fare,0.272376,-0.616062,0.228917,0.095978,0.199056,0.260993,1.000000


In [ ]:
#remove SibSp and Parch columns because they are not really related to Survived column
train_data_cpy.drop('SibSp', axis=1,inplace=True)
train_data_cpy.drop('Parch', axis=1,inplace=True)

3)	Learn and fine-tune a decision tree model

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn import tree

#splitting data for testing the model on the validation set
train_data_splitted, val_data = train_test_split(train_data_cpy,test_size=0.2,random_state=18)
#making x and y, the inputs of the model
train_set_x = train_data_splitted.drop("Survived",axis=1)
train_set_y = train_data_splitted["Survived"]

val_data_x = val_data.drop("Survived",axis=1)
val_data_y = val_data["Survived"]
#decision tree

max_depth = 5
min_sample = 50

decision_tree = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_sample)
decision_tree.fit(train_set_x,train_set_y)

DecisionTreeClassifier(max_depth=5, min_samples_split=50)

In [ ]:
#Accuracy of validation set
y_hat = decision_tree.predict(val_data_x)
accuracy_value = accuracy_score(val_data_y,y_hat)
print("The accuracy of the prediction is : ", accuracy_value)

The accuracy of the prediction is :  0.7535211267605634


In [ ]:
#plotting the decision tree
plt.figure(figsize=(30,25),dpi=500)
plot_tree(decision_tree, filled=True, feature_names=train_set_x.columns, class_names=['Survived', 'Not survived'])
plt.show()

4)	Apply the five-fold cross validation of your fine-tuned decision tree learning model to the Titanic training data to extract average classification accuracy

In [ ]:
from sklearn.model_selection import cross_val_score

new_x = train_data_cpy.drop(['Survived'], axis=1)
new_y = train_data_cpy['Survived']

# Apply five-fold cross validation
scores = cross_val_score(decision_tree, new_x, new_y, cv=5, scoring="accuracy")

print(f"Average accuracy: {scores.mean():.4f}")

Average accuracy: 0.7828


4.6)	Which algorithm is better, Decision Tree or Random Forest?

Random Forest is better than Decesion Tree because it gives higher score than decesion tree

4.7) Average score of random forest is higher and, standard deviation of decesion tree is lower.

5)	Apply the five-fold cross validation of your fine-tuned random forest learning  model to the Titanic training data to extract average classification accuracy

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()

# Apply five-fold cross validation
scores = cross_val_score(random_forest, new_x, new_y, cv=5, scoring="accuracy")

print(f"Average accuracy: {scores.mean():.4f}")

Average accuracy: 0.7842


Question-5: Build a bagging classifier using Scikit Learn for the above Titanic challenge

In [ ]:
from sklearn.ensemble import BaggingClassifier

# Initialize the BaggingClassifier
bagging_class = BaggingClassifier(base_estimator=decision_tree, n_estimators=20, random_state=42)

scores = cross_val_score(bagging_class, new_x, new_y, cv=5, scoring="accuracy")
print(f"Average accuracy: {scores.mean():.4f}")

Average accuracy: 0.7955


Question-6 : Build an Adaboost classifier using Scikit Learn for the above Titanic challenge.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

# Initialize the AdaBoostClassifier
adaboos_class = AdaBoostClassifier(base_estimator=decision_tree, n_estimators=20, random_state=42)

scores = cross_val_score(adaboos_class, new_x, new_y, cv=5, scoring="accuracy")
print(f"Average accuracy: {scores.mean():.4f}")

Average accuracy: 0.7899
